## BollingerBand_StrategyBacktesting

In [1]:
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd 
import pandas_datareader.data as pdr
import numpy as np
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

init_notebook_mode(connected = True)
cf.go_offline()

In [2]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

In [3]:
def bollinger_backtesting(ticker, previous_days):
    
    start = dt.datetime.today()-dt.timedelta(previous_days)
    end = dt.datetime.today()
    global ohlc_intraday
    ohlc_intraday = {}
    ohlc_intraday[ticker] = yf.download(ticker,start,end)
    ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
    global df_signal
    df_signal = ohlc_intraday[ticker].copy()
    df_signal = df_signal[['Date','Open','High','Low','Close']]
    
    df_signal['sma']= df_signal['Close'].rolling(20).mean()
    df_signal['std'] = df_signal['Close'].rolling(20).std()
    df_signal['upper']= df_signal['sma'] + (2*df_signal['std'])
    df_signal['lower']= df_signal['sma'] - (2*df_signal['std'])

    df_signal['Low_t-2']=df_signal['Low'].shift(2)
    df_signal['lower_t-2']=df_signal['lower'].shift(2)
    df_signal['Close_t-1']=df_signal['Close'].shift(1)
    df_signal['lower_t-1']=df_signal['lower'].shift(1)

    condition_t_2 = (df_signal['Low_t-2']<df_signal['lower_t-2'])
    condition_t_1 = (df_signal['Close_t-1']>df_signal['lower_t-1']) 
    buy_condition = condition_t_1 & condition_t_2
    df_signal['Buy']=np.where(buy_condition, 'Buy', '')
        
    df_signal = df_signal[['Date','Open','High','Low','Close','std','upper','lower','sma','Buy']]
    df_signal['Action'] = ''
    
    condition = (df_signal['Buy']=='Buy')
    df_signal2 = df_signal[condition].copy()
    df_signal2 = df_signal2.reset_index(drop = True)
    df_signal2 = df_signal2[['Date','Buy']]
    
    columns_Trading = ['Date_Buy','Price_Buy','Date_Sell','Price_Sell','PL(%)','Holding_Days','Accumulated profit']
    global df_Trading
    df_Trading = pd.DataFrame(columns = columns_Trading)
    
    Accu_profit = 0
    for i in range(len(df_signal2)):
        check_action = df_signal[df_signal['Date']==df_signal2['Date'][i]].reset_index()
        if check_action['Action'][0] =='':
            date_index = df_signal[df_signal['Date']==df_signal2['Date'][i]].index[0]
            Date_Buy = df_signal['Date'][date_index]
            Price_Buy = df_signal['Open'][date_index]
            stop_loss = 0.95*Price_Buy
            df_signal.loc[date_index,'Action']='Buy'
            date_index += 1
            stage = 'ToStop'
            close_order = False
            noskip = (date_index < len(df_signal))
            while True & noskip:
                df_signal['Action'][date_index] = 'Hold'
                if stage == 'ToStop':
                    if (df_signal['Close'][date_index] < stop_loss) |(df_signal['Close'][date_index] < df_signal['lower'][date_index]):
                        close_order = True
                        break
                    elif df_signal['Close'][date_index] > Price_Buy:
                        stage = 'ToProfit'
                elif stage == 'ToProfit':  
                    if (df_signal['High'][date_index-1] > df_signal['upper'][date_index-1])&(df_signal['Close'][date_index] < df_signal['upper'][date_index]):
                        close_order = True
                        break
                    elif df_signal['Close'][date_index] < Price_Buy:
                        stage = 'ToStop'
                date_index += 1
                if date_index >= len(df_signal):
                    break
            if close_order:
                date_index += 1
                df_signal['Action'][date_index] = 'Sell'
                Date_Sell = df_signal['Date'][date_index]
                Price_Sell = df_signal['Open'][date_index]
                fee = 0.002*(Price_Sell+Price_Buy)
                PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
                Holding_Days = abs(Date_Sell-Date_Buy).days
                Accu_profit += PL
                      
                Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,Holding_Days,Accu_profit]],columns = columns_Trading)
                df_Trading = df_Trading.append(Add_Trading)

    df_Trading = df_Trading.reset_index(drop = True)
    df_Trading = df_Trading.round(2)
    total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
    cagr = ((((total_profit+100)/100)**(1/(len(df_signal)/250)))-1)*100
    win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
    loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
    win_rate = win_trade*100/(win_trade+loss_trade)
    avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
    avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
    expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
    list_ = [["Total Profit",round(total_profit,2)],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],["Expected Return",str(round(expected_return,2))+'%']]
                      
    global df_summary
    df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])
                      
def plot_bollinger():

    fig = make_subplots(rows=3, cols=1,shared_xaxes=True, vertical_spacing=0.04,
    specs=[[{"type": "scatter"}],
          [{"type": "table"}],[{"type": "table"}]])

    buy = df_Trading[['Date_Buy','Price_Buy']].copy()
    sell = df_Trading[['Date_Sell','Price_Sell']].copy()
    
    fig.add_trace(go.Candlestick(x=df_signal['Date'],
                open=df_signal['Open'],
                high=df_signal['High'],
                low=df_signal['Low'],
                close=df_signal['Close']),row=1,col=1)

    #fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['Close'],mode='lines',name='Close'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['upper'],mode='lines',name='upper'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['lower'],mode='lines',name='lower'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['sma'],mode='lines',name='sma'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = buy['Date_Buy'], y=buy['Price_Buy'],mode='markers',name='buy'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = sell['Date_Sell'], y=sell['Price_Sell'],mode='markers',name='sell'),row= 1,col =1)
    
    fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=2, col=1)

    fig.add_trace(go.Table(header=dict(values=df_Trading.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_Trading[k].tolist() for k in df_Trading.columns],align = "left")),row=3, col=1)

    fig.update_layout( height=900, showlegend=True, title_text='Bollinger Band',)
    fig.update_layout(xaxis_rangeslider_visible=False)
    
    fig.show()

In [4]:
ticker = 'AMZN'
bollinger_backtesting(ticker, previous_days = 4000)

[*********************100%***********************]  1 of 1 completed


In [5]:
plot_bollinger()

In [6]:
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit
0,2011-06-08,187.45,2011-07-01,205.55,9.24,23,9.24
1,2011-08-08,196.40,2011-08-19,180.29,-8.59,11,0.65
2,2011-08-23,178.92,2011-09-20,240.80,34.12,28,34.77
3,2011-10-06,220.28,2011-10-18,242.31,9.58,12,44.35
4,2011-10-28,206.53,2011-11-22,186.95,-9.86,25,34.49
...,...,...,...,...,...,...,...
75,2021-05-10,3282.32,2021-05-13,3185.47,-3.34,3,135.79
76,2021-05-14,3185.56,2021-06-22,3458.06,8.14,39,143.93
77,2021-08-04,3379.35,2021-08-19,3194.02,-5.87,15,138.06
78,2021-10-01,3289.01,2021-10-05,3204.50,-2.96,4,135.09


In [7]:
# 1. Download Date from yfinance
ticker = 'AAPL'
start = dt.datetime.today()-dt.timedelta(4000)
end = dt.datetime.today()
ohlc_intraday = {}
ohlc_intraday[ticker] = yf.download(ticker,start,end)
ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
df_signal = ohlc_intraday[ticker].copy()
df_signal

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,10.144160,309080800
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,10.227141,255519600
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,10.218875,300428800
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,10.292056,311931200
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,10.485882,448560000
...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,179.449997,115228100
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,175.740005,153237000
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,174.330002,139380400
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,179.300003,131063300


In [8]:
# 2.remove some columns
df_signal = df_signal[['Date','Open','High','Low','Close']]
df_signal

,Date,Open,High,Low,Close
0,2011-01-04,11.872857,11.875000,11.719643,11.831786
1,2011-01-05,11.769643,11.940714,11.767857,11.928571
2,2011-01-06,11.954286,11.973214,11.889286,11.918929
3,2011-01-07,11.928214,12.012500,11.853571,12.004286
4,2011-01-10,12.101071,12.258214,12.041786,12.230357
...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003


In [9]:
# 3.Add std,upper band, lower band, sma 
df_signal['sma']= df_signal['Close'].rolling(20).mean()
df_signal['std'] = df_signal['Close'].rolling(20).std()
df_signal['upper']= df_signal['sma'] + (2*df_signal['std'])
df_signal['lower']= df_signal['sma'] - (2*df_signal['std'])
df_signal

,Date,Open,High,Low,Close,sma,std,upper,lower
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,NaN,NaN,NaN,NaN
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,NaN,NaN,NaN,NaN
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,NaN,NaN,NaN,NaN
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,NaN,NaN,NaN,NaN
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,162.279001,8.210813,178.700627,145.857375
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,163.566501,8.201276,179.969053,147.163948
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,164.783001,7.881234,180.545470,149.020532
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,166.198001,7.816619,181.831238,150.564764


In [10]:
# 4. Find long 'buy' day

df_signal['Low_t-2']=df_signal['Low'].shift(2)
df_signal['lower_t-2']=df_signal['lower'].shift(2)
df_signal['Close_t-1']=df_signal['Close'].shift(1)
df_signal['lower_t-1']=df_signal['lower'].shift(1)

condition_t_2 = (df_signal['Low_t-2']<df_signal['lower_t-2'])
condition_t_1 = (df_signal['Close_t-1']>df_signal['lower_t-1']) 

buy_condition = (condition_t_2) & (condition_t_1) 
df_signal['Buy']=np.where(buy_condition, 'Buy', '')

df_signal

,Date,Open,High,Low,Close,sma,std,upper,lower,Low_t-2,lower_t-2,Close_t-1,lower_t-1,Buy
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,NaN,NaN,NaN,NaN,NaN,NaN,11.831786,NaN,
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,NaN,NaN,NaN,NaN,11.719643,NaN,11.928571,NaN,
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,NaN,NaN,NaN,NaN,11.767857,NaN,11.918929,NaN,
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,NaN,NaN,NaN,NaN,11.889286,NaN,12.004286,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,162.279001,8.210813,178.700627,145.857375,170.699997,144.292139,174.559998,145.181977,
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,163.566501,8.201276,179.969053,147.163948,173.919998,145.181977,179.449997,145.857375,
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,164.783001,7.881234,180.545470,149.020532,174.690002,145.857375,175.740005,147.163948,
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,166.198001,7.816619,181.831238,150.564764,175.529999,147.163948,174.330002,149.020532,


In [11]:
# 5. remove some columns and add 'Action' column
df_signal = df_signal[['Date','Open','High','Low','Close','std','upper','lower','sma','Buy']]
df_signal['Action'] = ''
df_signal

,Date,Open,High,Low,Close,std,upper,lower,sma,Buy,Action
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,NaN,NaN,NaN,NaN,,
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,NaN,NaN,NaN,NaN,,
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,NaN,NaN,NaN,NaN,,
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,NaN,NaN,NaN,NaN,,
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,8.210813,178.700627,145.857375,162.279001,,
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,8.201276,179.969053,147.163948,163.566501,,
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,7.881234,180.545470,149.020532,164.783001,,
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,7.816619,181.831238,150.564764,166.198001,,


In [12]:
# 6. filter days 'buy' signal 
condition = (df_signal['Buy']=='Buy')
df_signal2 = df_signal[condition].copy()
df_signal2 = df_signal2.reset_index(drop = True)
df_signal2 = df_signal2[['Date','Buy']]
df_signal2

,Date,Buy
0,2011-03-18,Buy
1,2011-03-21,Buy
2,2011-03-22,Buy
3,2011-04-20,Buy
4,2011-05-18,Buy
...,...,...
155,2021-03-02,Buy
156,2021-05-07,Buy
157,2021-05-10,Buy
158,2021-05-14,Buy


In [13]:
# 7. Make trading dataframe
columns_Trading = ['Date_Buy','Price_Buy','Date_Sell','Price_Sell','PL(%)','Holding_Days','Accumulated profit']
df_Trading = pd.DataFrame(columns = columns_Trading)
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit


In [14]:
# 8.Trading loop and record in df_Trading
Accu_profit = 0
for i in range(len(df_signal2)):
    check_action = df_signal[df_signal['Date']==df_signal2['Date'][i]].reset_index()
    if check_action['Action'][0] =='':
        date_index = df_signal[df_signal['Date']==df_signal2['Date'][i]].index[0]
        Date_Buy = df_signal['Date'][date_index]
        Price_Buy = df_signal['Open'][date_index]
        stop_loss = 0.92*Price_Buy
        df_signal.loc[date_index,'Action']='Buy'
        date_index += 1
        stage = 'ToStop'
        close_order = False
        while True & (date_index < len(df_signal)):
            df_signal['Action'][date_index] = 'Hold'
            if stage == 'ToStop':
                if (df_signal['Close'][date_index] < stop_loss) | \
                (df_signal['Close'][date_index] < df_signal['lower'][date_index]):
                    close_order = True
                    break
                elif df_signal['Close'][date_index] > Price_Buy:
                    stage = 'ToProfit'
            elif stage == 'ToProfit':  
                if (df_signal['High'][date_index-1] > df_signal['upper'][date_index-1])& \
                (df_signal['Close'][date_index] < df_signal['upper'][date_index]):
                    close_order = True
                    break
                elif df_signal['Close'][date_index] < Price_Buy:
                    stage = 'ToStop'
            date_index += 1
            if date_index >= len(df_signal):
                break
        if close_order:
            date_index += 1
            df_signal['Action'][date_index] = 'Sell'
            Date_Sell = df_signal['Date'][date_index]
            Price_Sell = df_signal['Open'][date_index]
            fee = 0.002*(Price_Sell+Price_Buy)
            PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
            Holding_Days = abs(Date_Sell-Date_Buy).days
            Accu_profit += PL
            
            Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,
                                    Holding_Days,Accu_profit]],columns = columns_Trading)
            df_Trading = df_Trading.append(Add_Trading)
            
df_Trading = df_Trading.reset_index(drop = True)
df_Trading = df_Trading.round(2)
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit
0,2011-03-18,12.04,2011-06-13,11.69,-3.34,87,-3.34
1,2011-06-14,11.79,2011-06-21,11.31,-4.43,7,-7.77
2,2011-06-22,11.61,2011-07-12,12.63,8.31,20,0.54
3,2011-10-06,13.33,2011-12-29,14.41,7.64,84,8.18
4,2012-04-18,21.92,2012-04-23,20.38,-7.41,5,0.77
...,...,...,...,...,...,...,...
64,2021-02-22,128.01,2021-02-26,122.59,-4.63,4,-7.95
65,2021-03-01,123.75,2021-04-13,132.44,6.61,43,-1.34
66,2021-05-07,130.85,2021-05-11,123.50,-6.01,4,-7.35
67,2021-05-14,126.25,2021-06-10,127.02,0.21,27,-7.14


In [15]:
#9. Trading Summary
total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
cagr = ((((total_profit+100)/100)**(1/(len(df_signal)/250)))-1)*100
win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
win_rate = win_trade*100/(win_trade+loss_trade)
avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
list_ = [["Total Profit",round(total_profit,2)],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],["Expected Return",str(round(expected_return,2))+'%']]
                      
global df_summary
df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])
df_summary

,Measurement,Result
0,Total Profit,-3.92
1,CAGR,-0.36%
2,Win Trade,33
3,Loss Trade,36
4,Win Rate,47.83%
5,Average Gain,6.65%
6,Average loss,-6.2%
7,Expected Return,6.08%


In [16]:
#10.Plot graph

fig = make_subplots(rows=3, cols=1,shared_xaxes=True, vertical_spacing=0.04,
specs=[[{"type": "scatter"}],
          [{"type": "table"}],[{"type": "table"}]])

buy = df_Trading[['Date_Buy','Price_Buy']].copy()
sell = df_Trading[['Date_Sell','Price_Sell']].copy()
    
fig.add_trace(go.Candlestick(x=df_signal['Date'],
                open=df_signal['Open'],
                high=df_signal['High'],
                low=df_signal['Low'],
                close=df_signal['Close']),row=1,col=1)

    #fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['Close'],mode='lines',name='Close'),row= 1,col =1)
fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['upper'],mode='lines',name='upper'),row= 1,col =1)
fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['lower'],mode='lines',name='lower'),row= 1,col =1)
fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['sma'],mode='lines',name='sma'),row= 1,col =1)
fig.add_trace(go.Scatter(x = buy['Date_Buy'], y=buy['Price_Buy'],mode='markers',name='buy'),row= 1,col =1)
fig.add_trace(go.Scatter(x = sell['Date_Sell'], y=sell['Price_Sell'],mode='markers',name='sell'),row= 1,col =1)
    
fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=2, col=1)

fig.add_trace(go.Table(header=dict(values=df_Trading.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_Trading[k].tolist() for k in df_Trading.columns],align = "left")),row=3, col=1)

fig.update_layout( height=900, showlegend=True, title_text='Bollinger Band',)
fig.update_layout(xaxis_rangeslider_visible=False)
    
fig.show()